In [ ]:
!python get-pip.py

In [6]:
!pip --version

pip 25.0.1 from /home/quynd/vidCapLLM/venv/lib/python3.11/site-packages/pip (python 3.11)


In [2]:
!pip install -e .

Obtaining file:///home/quynd/vidCapLLM/DeepSeek-VL2
  Installing build dependencies ... one
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Building editable for deepseek_vl2 (pyproject.toml) ... done
  Created wheel for deepseek_vl2: filename=deepseek_vl2-1.0.0-0.editable-py3-none-any.whl size=14672 sha256=2ee53b8f7ddbf57221372daf4a9c504ceea09ed7379b05fc34fda02d077fe2f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-vwbizmsl/wheels/c7/3a/be/2df7ced65a0650263c4048d2af8f4963bf95998ca4e8f893a2
Successfully built deepseek_vl2
  Attempt

In [4]:
!python --version

Python 3.11.11


In [1]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images


# specify the path to the model
model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()


/home/quynd/vidCapLLM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python version is above 3.10, patching the collections module.


/home/quynd/vidCapLLM/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add video token = ['<video>'] to the tokenizer
<video>:128823
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



In [1]:
import numpy as np
## single image conversation example
## Please note that <|ref|> and <|/ref|> are designed specifically for the object localization feature. These special tokens are not required for normal conversations.
## If you would like to experience the grounded captioning functionality (responses that include both object localization and reasoning), you need to add the special token <|grounding|> at the beginning of the prompt. Examples could be found in Figure 9 of our paper.
conversation = [
    {
        "role": "<|User|>",
        "content": "Description for the video: <video>", 
    },
    {"role": "<|Assistant|>", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    video = torch.rand(4, 1280, dtype=torch.bfloat16),
    force_batchify=True,
    system_prompt=""
).to(vl_gpt.device)

print("prepare_inputs", prepare_inputs)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

print(inputs_embeds.shape)

# run the model to get the response
outputs = vl_gpt.language.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=False)
print(f"{prepare_inputs['sft_format'][0]}", answer)

NameError: name 'load_pil_images' is not defined

In [9]:
!gpustat

aircsrv5-Z690-Pro-RS              Thu Mar 13 15:46:16 2025  550.120
[0] NVIDIA GeForce RTX 4080 SUPER | 35°C,   4 % |   424 / 16376 MB | aircsrv5(249M) aircsrv5(71M) aircsrv5(64M) aircsrv5(18M)
